# NOTEBOOK 2 (Version optimisée faible RAM) : Fine-tuning de DistilRoBERTa

**Objectif :** Ce notebook entraîne un classifieur **léger** pour assigner des catégories aux dépôts GitHub, en utilisant des techniques d'optimisation pour fonctionner dans un environnement à **mémoire vive (RAM) limitée**.

In [ ]:
!pip install pandas numpy torch transformers datasets scikit-learn sentence-transformers accelerate bitsandbytes

In [ ]:
import pandas as pd
import numpy as np
import json
import gc
from tqdm.notebook import tqdm

# Hugging Face
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

# Métriques et Utilitaires
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# --- CONFIGURATION ---

# Fichiers d'entrée
INPUT_CSV_FILE = "github_data_with_readmes.csv"
CATEGORIES_FILE = "github_categories_database.json"

# Modèles
BASE_MODEL = 'distilroberta-base' # <--- MODÈLE PLUS LÉGER
EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'

# Paramètres
OUTPUT_MODEL_DIR = "./distilroberta_github_classifier"
TEST_SIZE = 0.2

# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Utilisation du device : {device}")

In [ ]:
# Chargement des données des dépôts
df = pd.read_csv(INPUT_CSV_FILE)
df['description'] = df['description'].fillna('')
df['readme_content'] = df['readme_content'].fillna('')
df['full_text'] = df['description'] + ' ' + df['readme_content']
df = df[df['full_text'].str.strip().str.len() > 50].reset_index(drop=True)

# Chargement de la base de catégories
with open(CATEGORIES_FILE, 'r', encoding='utf-8') as f:
    categories_db = json.load(f)

id2label = {cat['category_id']: cat['category_name'] for cat in categories_db}
label2id = {v: k for k, v in id2label.items()}
N_LABELS = len(categories_db)

print(f"{len(df)} dépôts et {N_LABELS} catégories chargés.")

In [ ]:
print("Création des étiquettes pour l'entraînement supervisé...")

embedding_model = SentenceTransformer(EMBEDDING_MODEL, device=device)
repo_embeddings = embedding_model.encode(
    df['full_text'].tolist(), 
    show_progress_bar=True,
    batch_size=32
)

category_embeddings = np.array([cat['embedding_prototype'] for cat in categories_db])

print("Assignation de la catégorie la plus proche à chaque dépôt...")
similarity_matrix = cosine_similarity(repo_embeddings, category_embeddings)
df['label'] = np.argmax(similarity_matrix, axis=1)

# --- Nettoyage Mémoire ---
# On supprime les gros objets dont on n'a plus besoin avant l'entraînement
del embedding_model, repo_embeddings, category_embeddings, similarity_matrix
gc.collect()
torch.cuda.empty_cache() if device == 'cuda' else None
print("Objets intermédiaires nettoyés de la mémoire.")

df.rename(columns={'full_text': 'text'}, inplace=True)
df = df[['text', 'label']] # On ne garde que les colonnes utiles
gc.collect()

In [ ]:
# Conversion en objet Dataset de Hugging Face
full_dataset = Dataset.from_pandas(df)

# Division en jeux d'entraînement et de test
hf_datasets = full_dataset.train_test_split(test_size=TEST_SIZE, stratify_by_column="label")

print(hf_datasets)

# Tokenisation
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

def tokenize_function(examples):
    # On utilise une troncature dynamique pour économiser de la mémoire
    return tokenizer(examples['text'], padding=True, truncation=True, max_length=512)

tokenized_datasets = hf_datasets.map(tokenize_function, batched=True)

In [ ]:
# Chargement du modèle pré-entraîné
model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL, 
    num_labels=N_LABELS,
    id2label=id2label,
    label2id=label2id
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_MODEL_DIR,
    
    # --- Optimisations Mémoire ---
    per_device_train_batch_size=4,        # 1. Très petite taille de lot
    gradient_accumulation_steps=8,        # 2. Accumuler les gradients pour simuler un lot de 4*8=32
    optim="adamw_8bit",                   # 3. Utiliser un optimiseur qui consomme 4x moins de mémoire
    gradient_checkpointing=True,          # 4. Échange un peu de temps de calcul contre beaucoup de mémoire
    fp16=True if device == 'cuda' else False, # 5. Utiliser des nombres à virgule flottante 16-bit (nécessite un GPU)

    # --- Paramètres classiques ---
    num_train_epochs=3,
    logging_steps=100,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

print("Lancement du fine-tuning optimisé...")
trainer.train()

In [ ]:
# --- ÉVALUATION FINALE ---
print("\nÉvaluation du modèle final sur le jeu de test...")
eval_results = trainer.evaluate()
print("\n--- RÉSULTATS DE L'ÉVALUATION ---")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")

trainer.save_model(OUTPUT_MODEL_DIR)
tokenizer.save_pretrained(OUTPUT_MODEL_DIR)
print(f"\n✅ Modèle fine-tuné et tokenizer sauvegardés dans '{OUTPUT_MODEL_DIR}'")

# --- TEST SUR UN NOUVEL EXEMPLE ---
from transformers import pipeline

classifier = pipeline("text-classification", model=OUTPUT_MODEL_DIR)
new_repo_description = "A lightweight CSS framework for minimalist and modern web design."
prediction = classifier(new_repo_description)

print("\n--- TEST SUR UN NOUVEL EXEMPLE ---")
print(f"Description: '{new_repo_description}'")
print(f"Catégorie prédite : {prediction[0]['label']} (Score: {prediction[0]['score']:.4f})")